<a href="https://colab.research.google.com/github/Minh-A/ARIMA_Dacon/blob/main/AutoARIMA%26ARIMA_Dacon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [ ]:
# 구글 코랩 연결
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Auto ARIMA Library 설치
!pip install pmdarima

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import matplotlib as mpl
import matplotlib.pyplot as plt 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA

In [ ]:
# 날짜 처리 ARIMQ용
data1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/여름방학 세미나(Dacon)/input/201901-202003.csv')
data1 = data1.fillna('')

In [ ]:
data1

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3
...,...,...,...,...,...,...,...,...,...,...,...,...
24697787,202003,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,3,43300,4
24697788,202003,충북,충주시,휴양콘도 운영업,충북,충주시,40s,1,3,3,35000,3
24697789,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,4,188000,6
24697790,202003,충북,충주시,휴양콘도 운영업,충북,충주시,50s,2,4,4,99000,6


In [ ]:
# 2020 4월 데이터
data11 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/여름방학 세미나(Dacon)/input/202004.csv')
data11 = data11.fillna('')

In [ ]:
data11

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,3,779600,4
1,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,2,3,14,2427700,15
2,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,50s,1,4,18,2184600,18
3,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,50s,2,4,21,3653100,23
4,202004,강원,강릉시,건강보조식품 소매업,강원,강릉시,60s,1,5,15,2765500,18
...,...,...,...,...,...,...,...,...,...,...,...,...
1350317,202004,충북,충주시,휴양콘도 운영업,충북,충주시,20s,1,1,5,77000,5
1350318,202004,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,6,92000,6
1350319,202004,충북,충주시,휴양콘도 운영업,충북,충주시,40s,2,3,5,193000,5
1350320,202004,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,5,86000,7


# 데이터 셋 생성

In [ ]:
# 데이터1 정제
data1_1 = data1.drop(['CARD_CCG_NM', 'HOM_CCG_NM','FLC','HOM_SIDO_NM','AGE','SEX_CTGO_CD','CSTMR_CNT','CNT'], axis=1)

columns = ['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM']
data1_1 = data1_1.groupby(columns).sum().reset_index(drop=False)
data1_1

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,강원,건강보조식품 소매업,148354746
1,201901,강원,골프장 운영업,1229200312
2,201901,강원,과실 및 채소 소매업,1286979106
3,201901,강원,관광 민예품 및 선물용품 소매업,30656770
4,201901,강원,그외 기타 스포츠시설 운영업,13101500
...,...,...,...,...
9433,202003,충북,피자 햄버거 샌드위치 및 유사 음식점업,1315245299
9434,202003,충북,한식 음식점업,16152482704
9435,202003,충북,호텔업,15248550
9436,202003,충북,화장품 및 방향제 소매업,428881434


In [ ]:
# 데이터2 정제
data11_1 = data11.drop(['CARD_CCG_NM', 'HOM_CCG_NM','FLC','HOM_SIDO_NM','AGE','SEX_CTGO_CD','CSTMR_CNT','CNT'], axis=1)

columns = ['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM']
data11_1 = data11_1.groupby(columns).sum().reset_index(drop=False)
data11_1

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,88823988
1,202004,강원,골프장 운영업,4708346820
2,202004,강원,과실 및 채소 소매업,1121028924
3,202004,강원,관광 민예품 및 선물용품 소매업,14360780
4,202004,강원,그외 기타 스포츠시설 운영업,227200
...,...,...,...,...
605,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,1373635928
606,202004,충북,한식 음식점업,18911036160
607,202004,충북,호텔업,14121500
608,202004,충북,화장품 및 방향제 소매업,450507431


In [ ]:
# 데이터1과 데이터2를 합치기
data2 = pd.concat([data1_1, data11_1], axis=0)
data2 = data2.reset_index(drop=True)
data2

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,강원,건강보조식품 소매업,148354746
1,201901,강원,골프장 운영업,1229200312
2,201901,강원,과실 및 채소 소매업,1286979106
3,201901,강원,관광 민예품 및 선물용품 소매업,30656770
4,201901,강원,그외 기타 스포츠시설 운영업,13101500
...,...,...,...,...
10043,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,1373635928
10044,202004,충북,한식 음식점업,18911036160
10045,202004,충북,호텔업,14121500
10046,202004,충북,화장품 및 방향제 소매업,450507431


In [ ]:
# 제공 데이터 인코딩
dtypes = data2.dtypes
encoders = {}
for column in data2.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(data2[column])
        encoders[column] = encoder
        
for column in encoders.keys():
    encoder = encoders[column]
    data2[column] = encoder.transform(data2[column])

In [ ]:
data2

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,0,0,148354746
1,201901,0,1,1229200312
2,201901,0,2,1286979106
3,201901,0,3,30656770
4,201901,0,5,13101500
...,...,...,...,...
10043,202004,16,36,1373635928
10044,202004,16,37,18911036160
10045,202004,16,38,14121500
10046,202004,16,39,450507431


In [ ]:
# 템플릿 생성
REG_YYMMs     = data2['REG_YYMM'].unique()
CARD_SIDO_NMs = data2['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = data2['STD_CLSS_NM'].unique()
STD_CLSS_NMs.sort()                                    # 매장 순서정렬을 위한 코드

temp = []
for REG_YYMM in REG_YYMMs:
    for CARD_SIDO_NM in CARD_SIDO_NMs:
        for STD_CLSS_NM in STD_CLSS_NMs:
            temp.append([REG_YYMM, CARD_SIDO_NM, STD_CLSS_NM])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns= ['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])

In [ ]:
temp

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM
0,201901,0,0
1,201901,0,1
2,201901,0,2
3,201901,0,3
4,201901,0,4
...,...,...,...
11147,202004,16,36
11148,202004,16,37
11149,202004,16,38
11150,202004,16,39


In [ ]:
# 기초 데이터셋 생성
daset = pd.merge(temp, data2, how= 'left', left_on = ('REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'), right_on = ('REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'))

In [ ]:
daset

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,0,0,1.483547e+08
1,201901,0,1,1.229200e+09
2,201901,0,2,1.286979e+09
3,201901,0,3,3.065677e+07
4,201901,0,4,NaN
...,...,...,...,...
11147,202004,16,36,1.373636e+09
11148,202004,16,37,1.891104e+10
11149,202004,16,38,1.412150e+07
11150,202004,16,39,4.505074e+08


In [ ]:
# 결측치에 0을 넣기
daset=daset.fillna(0)
daset

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,201901,0,0,1.483547e+08
1,201901,0,1,1.229200e+09
2,201901,0,2,1.286979e+09
3,201901,0,3,3.065677e+07
4,201901,0,4,0.000000e+00
...,...,...,...,...
11147,202004,16,36,1.373636e+09
11148,202004,16,37,1.891104e+10
11149,202004,16,38,1.412150e+07
11150,202004,16,39,4.505074e+08


In [ ]:
# 메인 키값 설정
daset['key'] = daset['CARD_SIDO_NM']*100 + daset['STD_CLSS_NM']
daset["AMT"] = np.log1p(daset['AMT'])
daset = daset[["key","AMT"]]
daset

,key,AMT
0,0,18.815117
1,1,20.929630
2,2,20.975564
3,3,17.238364
4,4,0.000000
...,...,...
11147,1636,21.040727
11148,1637,23.663012
11149,1638,16.463209
11150,1639,19.925885


# 예측 코드

## 5월 예측 템플릿 만들기

In [ ]:
# 템플릿 만들기
newdata5 = daset.loc[daset['key']== 4]
newdata5 = newdata5.reset_index(drop= True)

In [ ]:
newdata5

,key,AMT
0,4,0.000000
1,4,0.000000
2,4,0.000000
3,4,0.000000
4,4,12.083911
5,4,0.000000
6,4,0.000000
7,4,0.000000
8,4,0.000000
9,4,0.000000


In [ ]:
# 템플릿 분할
newdata6 = newdata5.copy()
newdata7 = newdata5.copy()

In [ ]:
# 5월 예측 템플릿
## from statsmodels.tsa.arima_model import ARIMA의 ARIMA 사용
for aa in range(17):
  for i in range(41):
    num = num = aa*100 + i
    print(num)
    ii = daset.loc[daset['key']==num]
    ii = ii.reset_index(drop= True)
    if ii["AMT"][0] == 0:
      print("결측치 있어")
      ii.loc[10000] = [num, 0]
      newdata5 = pd.concat([newdata5, ii], axis=0)
    else:  
      arma = ARIMA(ii["AMT"], order = (0,0,0))
      arima = arma.fit(trend = 'c', full_output = True, disp=1)
      bb = arima.forecast(steps=1)
      print(bb[0][0])
      ii.loc[10000] = [num, bb[0][0]]
      newdata5 = pd.concat([newdata5, ii], axis=0)

In [ ]:
# 5월 예측 템플릿
## pmdarima의 Auto Arima 사용
for aa in range(17):
  for i in range(41):
    num = num = aa*100 + i
    print(num)
    ii = daset.loc[daset['key']==num]
    ii = ii.reset_index(drop= True)
    if ii["AMT"][0] == 0:
      print("결측치 있어")
      ii.loc[10000] = [num, 0]
      newdata5 = pd.concat([newdata5, ii], axis=0)
    else:  
      arima = auto_arima(ii["AMT"],start_p=0, d=0, start_q=0, max_p=10, max_d=3, max_q=10, 
                         start_P=0, D= 0, start_Q=0, 
                         max_P=5, max_D=3, max_Q=5, max_order=5, m=1, 
                         seasonal= True, stationary=False, information_criterion='aic', alpha=0.05, test='kpss', 
                         seasonal_test='ocsb', stepwise=False, n_jobs=-1, start_params=None, trend=None, method='powell', 
                         maxiter=12, offset_test_args=None, seasonal_test_args=None, suppress_warnings=False, 
                         error_action='trace', trace=1, 
                         scoring='mse', scoring_args=None, with_intercept="auto")
      bb = arima.predict(n_periods=1)
      print(bb[0])
      ii.loc[10000] = [num, bb[0]]
      newdata5 = pd.concat([newdata5, ii], axis=0)

In [ ]:
# 2020년 5월 예측 템플릿
newdata5 = newdata5.iloc[16:,:].reset_index(drop= True)
newdata5

,key,AMT
0,0.0,18.815117
1,0.0,18.804008
2,0.0,18.510878
3,0.0,18.484111
4,0.0,18.813505
...,...,...
11844,1640.0,19.218517
11845,1640.0,17.667483
11846,1640.0,16.359746
11847,1640.0,16.048576


## 6월 예측 템플릿 만들기

In [ ]:
# 6월 예측
## from statsmodels.tsa.arima_model import ARIMA의 ARIMA 사용
for aa in range(17):
  for i in range(41):
    num = num = aa*100 + i
    ii = newdata5.loc[newdata5['key']==num]
    ii = ii.reset_index(drop= True)
    if ii["AMT"][0] == 0:
      ii.loc[10000] = [num, 0]
      newdata6 = pd.concat([newdata6, ii], axis=0)
    else:  
      arma = ARIMA(ii["AMT"], order = (0,0,0))
      arima = arma.fit(trend = 'c', full_output = True, disp=1)
      bb = arima.forecast(steps=1)
      ii.loc[10000] = [num, bb[0][0]]
      newdata6 = pd.concat([newdata6, ii], axis=0)

In [ ]:
# 6월 예측
## pmdarima의 Auto Arima 사용
for aa in range(17):
  for i in range(41):
    num = num = aa*100 + i
    ii = newdata5.loc[newdata5['key']==num]
    ii = ii.reset_index(drop= True)
    if ii["AMT"][0] == 0:
      ii.loc[10000] = [num, 0]
      newdata6 = pd.concat([newdata6, ii], axis=0)
    else:  
      arima = auto_arima(ii["AMT"],start_p=0, d=0, start_q=0, max_p=10, max_d=3, max_q=10, 
                         start_P=0, D= 0, start_Q=0, 
                         max_P=5, max_D=3, max_Q=5, max_order=5, m=1, 
                         seasonal= True, stationary=False, information_criterion='aic', alpha=0.05, test='kpss', 
                         seasonal_test='ocsb', stepwise=False, n_jobs=-1, start_params=None, trend=None, method='powell', 
                         maxiter=12, offset_test_args=None, seasonal_test_args=None, suppress_warnings=False, 
                         error_action='trace', trace=1, 
                         scoring='mse', scoring_args=None, with_intercept="auto")
      bb = arima.predict(n_periods=1)
      print(bb[0])
      ii.loc[10000] = [num, bb[0]]
      newdata6 = pd.concat([newdata6, ii], axis=0)

In [ ]:
# 2020년 6월 예측 템플릿
newdata6 = newdata6.iloc[16:,:].reset_index(drop= True)
newdata6

,key,AMT
0,0.0,18.815117
1,0.0,18.804008
2,0.0,18.510878
3,0.0,18.484111
4,0.0,18.813505
...,...,...
12541,1640.0,17.667483
12542,1640.0,16.359746
12543,1640.0,16.048576
12544,1640.0,18.504392


## 7월 예측 템플릿 만들기 & CSV

In [ ]:
# 제출 CSV용 템플릿 만들기
temp7e = daset.loc[daset['key']== 4]
temp7e = temp7e.reset_index(drop= True)

In [ ]:
# 7월 예측
## from statsmodels.tsa.arima_model import ARIMA의 ARIMA 사용
for aa in range(17):
  for i in range(41):
    num = num = aa*100 + i
    ii = newdata6.loc[newdata6['key']==num]
    ii = ii.reset_index(drop= True)
    if ii["AMT"][0] == 0:
      ii.loc[10000] = [num, 0]
      newdata7 = pd.concat([newdata7, ii], axis=0)
      temp7e.loc[num] = [num, 1]
    else:  
      arma = ARIMA(ii["AMT"], order = (0,0,0))
      arima = arma.fit(trend = 'c', full_output = True, disp=1)
      bb = arima.forecast(steps=1)
      ii.loc[10000] = [num, bb[0][0]]
      newdata7 = pd.concat([newdata7, ii], axis=0)
      temp7e.loc[num] = [num, bb[0][0]]

In [ ]:
# 7월 예측
## pmdarima의 Auto Arima 사용
for aa in range(17):
  for i in range(41):
    num = num = aa*100 + i
    ii = newdata6.loc[newdata6['key']==num]
    ii = ii.reset_index(drop= True)
    if ii["AMT"][0] == 0:
      ii.loc[10000] = [num, 0]
      newdata7 = pd.concat([newdata7, ii], axis=0)
      temp7e.loc[num] = [num, 1]
    else:  
      arima = auto_arima(ii["AMT"],start_p=0, d=0, start_q=0, max_p=10, max_d=3, max_q=10, 
                         start_P=0, D= 0, start_Q=0, 
                         max_P=5, max_D=3, max_Q=5, max_order=5, m=1, 
                         seasonal= True, stationary=False, information_criterion='aic', alpha=0.05, test='kpss', 
                         seasonal_test='ocsb', stepwise=False, n_jobs=-1, start_params=None, trend=None, method='powell', 
                         maxiter=12, offset_test_args=None, seasonal_test_args=None, suppress_warnings=False, 
                         error_action='trace', trace=1, 
                         scoring='mse', scoring_args=None, with_intercept="auto")
      bb = arima.predict(n_periods=1)
      print(bb[0])
      ii.loc[10000] = [num, bb[0]]
      newdata7 = pd.concat([newdata7, ii], axis=0)
      temp7e.loc[num] = [num, bb[0][0]]

In [ ]:
# 2020년 7월 예측값
newdata7 = newdata7.iloc[16:,:].reset_index(drop= True)
newdata7

,key,AMT
0,0.0,18.815117
1,0.0,18.804008
2,0.0,18.510878
3,0.0,18.484111
4,0.0,18.813505
...,...,...
13238,1640.0,16.359746
13239,1640.0,16.048576
13240,1640.0,18.504392
13241,1640.0,18.504392


In [ ]:
# 2020년 7월만의 예측값
temp7e

,key,AMT
0,0.0,18.558411
1,1.0,21.793198
2,2.0,20.814153
3,3.0,17.116711
4,4.0,1.000000
...,...,...
1636,1636.0,21.068642
1637,1637.0,23.790600
1638,1638.0,17.254635
1639,1639.0,20.203647


In [ ]:
# 로그값을 지수값으로 변환
temp7e["AMT"] = np.expm1(temp7e["AMT"])

In [ ]:
# 제출용 파일 완성
temp7e

,key,AMT
0,0.0,1.147665e+08
1,1.0,2.915183e+09
2,2.0,1.095145e+09
3,3.0,2.714520e+07
4,4.0,1.718282e+00
...,...,...
1636,1636.0,1.412522e+09
1637,1637.0,2.148454e+10
1638,1638.0,3.115968e+07
1639,1639.0,5.947473e+08


# 4월과 7월 데이터만 뽑기

## 4월 데이터 넣기 (7월 28일 제공)

In [ ]:
# 제공 데이터 인코딩
dtypes = data11_1.dtypes
encoders = {}
for column in data11_1.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(data11_1[column])
        encoders[column] = encoder
        
for column in encoders.keys():
    encoder = encoders[column]
    data11_1[column] = encoder.transform(data11_1[column])

In [ ]:
data11_1

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,0,0,88823988
1,202004,0,1,4708346820
2,202004,0,2,1121028924
3,202004,0,3,14360780
4,202004,0,5,227200
...,...,...,...,...
605,202004,16,36,1373635928
606,202004,16,37,18911036160
607,202004,16,38,14121500
608,202004,16,39,450507431


In [ ]:
# 4월 템플릿 생성
REG_YYMMs     = [202004]
CARD_SIDO_NMs = data2['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = data2['STD_CLSS_NM'].unique()
STD_CLSS_NMs.sort()                                  

temp4 = []
for REG_YYMM in REG_YYMMs:
    for CARD_SIDO_NM in CARD_SIDO_NMs:
        for STD_CLSS_NM in STD_CLSS_NMs:
            temp4.append([REG_YYMM, CARD_SIDO_NM, STD_CLSS_NM])
temp4 = np.array(temp4)
temp4 = pd.DataFrame(data=temp4, columns= ['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'])

In [ ]:
temp4

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM
0,202004,0,0
1,202004,0,1
2,202004,0,2
3,202004,0,3
4,202004,0,4
...,...,...,...
692,202004,16,36
693,202004,16,37
694,202004,16,38
695,202004,16,39


In [ ]:
# temp4 생성, 결측치에 0을 넣음
temp4 = pd.merge(temp4, data11_1, how= 'left', left_on = ('REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'), right_on = ('REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM'))
temp4 = temp4.fillna(0)

In [ ]:
temp4

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,0,0,8.882399e+07
1,202004,0,1,4.708347e+09
2,202004,0,2,1.121029e+09
3,202004,0,3,1.436078e+07
4,202004,0,4,0.000000e+00
...,...,...,...,...
692,202004,16,36,1.373636e+09
693,202004,16,37,1.891104e+10
694,202004,16,38,1.412150e+07
695,202004,16,39,4.505074e+08


## 7월 데이터 예측

In [ ]:
# 템플릿 만들기
temp7 = daset.loc[daset['key']== 4]
temp7 = temp7.reset_index(drop= True)

In [ ]:
# 7월 예측 코드
## from statsmodels.tsa.arima_model import ARIMA의 ARIMA 사용
for aa in range(17):
  for i in range(41):
    num = num = aa*100 + i
    print(num)
    ii = newdata6.loc[newdata6['key']==num]
    ii = ii.reset_index(drop= True)
    if ii["AMT"][0] == 0:
      print("결측치 있어")
      temp7.loc[num] = [num, 1]
    else:  
      arma = ARIMA(ii["AMT"], order = (0,0,0))
      arima = arma.fit(trend = 'c', full_output = True, disp=1)
      bb = arima.forecast(steps=1)
      print(bb[0][0])
      temp7.loc[num] = [num, bb[0][0]]

In [ ]:
# 7월 예측 코드
## pmdarima의 Auto Arima 사용
for aa in range(17):
  for i in range(41):
    num = num = aa*100 + i
    print(num)
    ii = newdata6.loc[newdata6['key']==num]
    ii = ii.reset_index(drop= True)
    if ii["AMT"][0] == 0:
      print("결측치 있어")
      temp7.loc[num] = [num, 1]
    else:  
      arima = auto_arima(ii["AMT"],start_p=0, d=0, start_q=0, max_p=10, max_d=3, max_q=10, 
                         start_P=0, D= 0, start_Q=0, 
                         max_P=5, max_D=3, max_Q=5, max_order=5, m=1, 
                         seasonal= True, stationary=False, information_criterion='aic', alpha=0.05, test='kpss', 
                         seasonal_test='ocsb', stepwise=False, n_jobs=-1, start_params=None, trend=None, method='powell', 
                         maxiter=12, offset_test_args=None, seasonal_test_args=None, suppress_warnings=False, 
                         error_action='trace', trace=1, 
                         scoring='mse', scoring_args=None, with_intercept="auto")
      bb = arima.predict(n_periods=1)
      print(bb[0])
      temp7.loc[num] = [num, bb[0]]

In [ ]:
# 7월 예측
temp7

,key,AMT
0,0.0,18.558411
1,1.0,21.793198
2,2.0,20.814153
3,3.0,17.116711
4,4.0,1.000000
...,...,...
1636,1636.0,21.068642
1637,1637.0,23.790600
1638,1638.0,17.254635
1639,1639.0,20.203647


In [ ]:
# 로그값을 지수값으로 변환
temp7["AMT"] = np.expm1(temp7["AMT"])

In [ ]:
temp7

,key,AMT
0,0.0,1.147665e+08
1,1.0,2.915183e+09
2,2.0,1.095145e+09
3,3.0,2.714520e+07
4,4.0,1.718282e+00
...,...,...
1636,1636.0,1.412522e+09
1637,1637.0,2.148454e+10
1638,1638.0,3.115968e+07
1639,1639.0,5.947473e+08


# 데이콘 제출

In [ ]:
# 4월과 7월 합치기
dacon =  pd.concat([temp4, temp7], axis=0)
dacon = dacon.reset_index(drop= True)

In [ ]:
# 중요한 것은 AMT부분!!
dacon

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT,key
0,202004.0,0.0,0.0,8.882399e+07,NaN
1,202004.0,0.0,1.0,4.708347e+09,NaN
2,202004.0,0.0,2.0,1.121029e+09,NaN
3,202004.0,0.0,3.0,1.436078e+07,NaN
4,202004.0,0.0,4.0,0.000000e+00,NaN
...,...,...,...,...,...
1389,NaN,NaN,NaN,1.412522e+09,1636.0
1390,NaN,NaN,NaN,2.148454e+10,1637.0
1391,NaN,NaN,NaN,3.115968e+07,1638.0
1392,NaN,NaN,NaN,5.947473e+08,1639.0


In [ ]:
# 반올림
dacon['AMT'] = np.round(dacon["AMT"], 0)

In [ ]:
# 제출 데이터 불러오기
submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/여름방학 세미나(Dacon)/input/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)

In [ ]:
# 제출 데이터랑 합치기
submission =  pd.concat([submission, dacon["AMT"]], axis=1)
submission.index.name = 'id'

In [ ]:
submission

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,8.882399e+07
1,202004,강원,골프장 운영업,4.708347e+09
2,202004,강원,과실 및 채소 소매업,1.121029e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.436078e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.412522e+09
1390,202007,충북,한식 음식점업,2.148454e+10
1391,202007,충북,호텔업,3.115968e+07


In [ ]:
# 파일 내보내기
submission.to_csv( '/content/drive/My Drive/Colab Notebooks/여름방학 세미나(dacon)/input/ARIMA_submission_7.csv', encoding='utf-8-sig')
submission

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,8.882399e+07
1,202004,강원,골프장 운영업,4.708347e+09
2,202004,강원,과실 및 채소 소매업,1.121029e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.436078e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.412522e+09
1390,202007,충북,한식 음식점업,2.148454e+10
1391,202007,충북,호텔업,3.115968e+07
